In [1]:
import numpy as np
import pandas as pd

In [2]:
raw_data = pd.read_csv("labeledTrainData.tsv",sep='\t',encoding="latin-1")
raw_data.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [14]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopword = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub("<[^>]>","",text) # Your code here
    
    # Save emoticons for later appending
    # Your code here
    emotions = re.findall("(?::|=|;)(?:-)?(?:\)|\(|D|P)",text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub("[\W]"," ",text.lower()) + " " + ' '.join(emotions)).replace("-","")
    
    return text
print(preprocessor("I like :) it :) "))

i like    it     :) :)


In [23]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here"I like :) it :) "
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()] # Your code here
tokenizer_porter(preprocessor("I like :) it :) "))

['i', 'like', 'it', ':)', ':)']

In [19]:
from sklearn.model_selection import train_test_split
X = raw_data["review"]
y = raw_data["sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = stopword, tokenizer = tokenizer_porter, preprocessor = preprocessor)# Your code here

clf = Pipeline([("vector",tfidf),("clf",LogisticRegression(random_state = 0))])
clf.fit(X_train, y_train)

C:\Users\ADMIN\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\ADMIN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vector',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x0000019A12143488>,
                                 smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'mys...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x0000019A122AD268>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
      

In [21]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
# Using Test dataset to evaluate model
predictions = clf.predict(X_test)
# classification_report
print("accuracy: ",accuracy_score(y_test,predictions))
print("classification_report \n", classification_report(y_test,predictions))
# confusion matrix
print("confusion matrix: \n", confusion_matrix(y_test,predictions))

accuracy:  0.8884
classification_report 
               precision    recall  f1-score   support

           0       0.90      0.87      0.89      2481
           1       0.88      0.91      0.89      2519

    accuracy                           0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000

confusion matrix: 
 [[2160  321]
 [ 237 2282]]
